In [142]:
import pandas as pd

from collections import deque
from uuid import uuid4

In [5]:
columns = [
    "player_id", 
    "position_id", 
    "name", 
    "season", 
    "week", 
    "fanduel_price",
    "fanduel_points",
    "proj_mean_fanduel_points",
    "proj_downside_fanduel_points",
    "proj_upside_fanduel_points",
]
df = pd.read_csv("../data/draftkings/player_data.csv", usecols=columns)

In [6]:
df = df.dropna(subset=["fanduel_price", "fanduel_points"]).sort_values(["season", "week"])
df = df[df["proj_mean_fanduel_points"] > 0]

In [4]:
POSITIONS = ["QB", "RB", "WR", "TE"]

In [ ]:
df = df[df.position_id.isin(POSITIONS)]

In [8]:
point_mask = df.groupby("player_id")["fanduel_points"].shift(1) > 3

In [9]:
df = df[point_mask]

In [64]:
inequality?

Docstring:
A utility function that can be used to declare inequality and
ranged inequality expressions.  The expression::

    inequality(2, model.x)

is equivalent to the expression::

    2 <= model.x

The expression::

    inequality(2, model.x, 3)

is equivalent to the expression::

    2 <= model.x <= 3

.. note:: This ranged inequality syntax is deprecated in Pyomo.
    This function provides a mechanism for expressing
    ranged inequalities without chained inequalities.

Args:
    lower: an expression defines a lower bound
    body: an expression defines the body of a ranged constraint
    upper: an expression defines an upper bound
    strict (bool): A boolean value that indicates whether the inequality
        is strict.  Default is :const:`False`.

Returns:
    A relational expression.  The expression is an inequality
    if any of the values :attr:`lower`, :attr:`body` or
    :attr:`upper` is :const:`None`.  Otherwise, the expression
    is a ranged inequality.
Type:      b

In [193]:
from pyomo.opt import SolverFactory
from pyomo.environ import (
    AbstractModel, 
    Set, 
    Param, 
    Boolean, 
    Objective, 
    Constraint, 
    Var, 
    summation, 
    maximize,
    inequality,
)

model = AbstractModel()

model.players = Set()

model.prices = Param(model.players)
model.points = Param(model.players)
model.positions = Param(model.players, range(len(POSITIONS)))

model.x = Var(model.players, domain=Boolean)

def obj_expression(model):
    return summation(model.points, model.x)

model.OBJ = Objective(rule=obj_expression, sense=maximize)

def budget_constraint(model):
    return sum(model.prices[i] * model.x[i] for i in model.players) <= 56000

def qb_constraint(model):
    return sum(model.positions[i, 0] * model.x[i] for i in model.players) == 1

def rb_constraint(model):
    return inequality(2, sum(model.positions[i, 1] * model.x[i] for i in model.players), 3)

def wr_constraint(model):
    return inequality(3, sum(model.positions[i, 2] * model.x[i] for i in model.players), 4)

def te_constraint(model):
    return inequality(1, sum(model.positions[i, 3] * model.x[i] for i in model.players), 2)

def flex_constraint(model):
    return sum(model.positions[i, p] * model.x[i] for i in model.players for p in range(1, 4)) == 7

model.BudgetConstraint = Constraint(rule=budget_constraint)
model.QBConstraint = Constraint(rule=qb_constraint)
model.RBConstraint = Constraint(rule=rb_constraint)
model.WRConstraint = Constraint(rule=wr_constraint)
model.TEConstraint = Constraint(rule=te_constraint)
model.FlexConstraint = Constraint(rule=flex_constraint)

# Real Deal Holyfield

In [112]:
df = pd.read_csv("../data/draftkings/slates/fd_slate_and_preds.csv")

In [113]:
df.tournament_name.value_counts()

20-Player League ($0)                                         1062
10-Player League ($0)                                          574
$10K Sun NFL Super Snap ($2K to 1st)                           257
$5K Sun NFL Super Snap ($1K to 1st)                            257
$3.3M NFL Sunday Million ($1M to 1st)                          239
$50K Sun NFL Kickoff ($10K to 1st)                             239
$25K Sun NFL Blitz ($5K to 1st)                                239
$600K Sun NFL Bomb ($100K to 1st)                              239
$2M WFFC Qualifier #2                                          239
$150K Sun NFL Rush ($30K to 1st)                               239
$100K Sun NFL Snap ($10K to 1st)                               239
$50K Sun NFL Rush ($10K to 1st)                                239
$33K Sun NFL Pooch Punt                                        239
$200K Sun NFL Huddle (Experienced Players Excluded)            239
$150K Sun NFL Kickoff ($30K to 1st)                           

In [116]:
tournament_df = df[df["tournament_name"] == "$1M NFL Monday Million ($200K to 1st)"]
tournament_df = tournament_df[tournament_df["rank"] == 1]

In [117]:
tournament_df

,player_id,season,week,name,pts,expected_start_date,total_spots,entry_fee,guaranteed_prizes,tournament_name,position,salary,current_team_id,franchise_id,team_name,rank,game_time
6,27,2020,1,Ben Roethlisberger,14.7916,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),QB,7800,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-14 19:15:00 EDT
445,1960,2020,1,Golden Tate,8.8870,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),WR,5600,nyg2020,NYG,New York Giants,1.0,2020-09-14 19:15:00 EDT
1291,3485,2020,1,Eric Ebron,7.0710,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),TE,5000,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-14 19:15:00 EDT
1719,3848,2020,1,Melvin Gordon,13.7840,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),RB,6600,den2020,DEN,Denver Broncos,1.0,2020-09-14 22:10:00 EDT
2469,4232,2020,1,Derrick Henry,15.5550,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),RB,8400,oti2020,OTI,Tennessee Titans,1.0,2020-09-14 22:10:00 EDT
2700,4481,2020,1,Kalif Raymond,2.0560,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),WR,4600,oti2020,OTI,Tennessee Titans,1.0,2020-09-14 22:10:00 EDT
2868,4597,2020,1,JuJu Smith-Schuster,12.8810,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),WR,7100,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-14 19:15:00 EDT
2900,4603,2020,1,James Conner,10.1250,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),RB,6800,pit2020,PIT,Pittsburgh Steelers,1.0,2020-09-14 19:15:00 EDT
2987,4631,2020,1,Jonnu Smith,3.0510,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),TE,4900,oti2020,OTI,Tennessee Titans,1.0,2020-09-14 22:10:00 EDT
2996,4632,2020,1,Corey Davis,4.1220,2020-09-14 23:10:00.000000,132275,9.0,1000000.0,$1M NFL Monday Million ($200K to 1st),WR,5000,oti2020,OTI,Tennessee Titans,1.0,2020-09-14 22:10:00 EDT


In [194]:
lineups = []
injuries = ["Courtland Sutton", "Golden Tate"]
ignore_players = deque(maxlen=2)
for i in range(10):
    player_pool = tournament_df[(~tournament_df.name.isin(ignore_players)) & (~tournament_df.name.isin(injuries))]
    player_prices = {i: price for i, price in enumerate(player_pool.salary.tolist())}
    player_points = {i: points for i, points in enumerate(player_pool.pts.tolist())}
    positions = pd.get_dummies(player_pool.position.astype(pd.CategoricalDtype(categories=POSITIONS))).values
    positions = {(i, p): positions[i, p] 
                 for i in range(positions.shape[0]) 
                 for p in range(positions.shape[1])}
    players = list(range(len(player_prices)))

    data = {
        None: {
            "players": {None: players},
            "prices": player_prices,
            "points": player_points,
            "positions": positions
        }, 
    }
    instance = model.create_instance(data=data)
    opt = SolverFactory('glpk')
    optimizer_result = opt.solve(instance)
    lineup = player_pool[[value > 0 for value in instance.x.get_values().values()]]
    lineup["lineup_id"] = uuid4().hex
    lineups.append(lineup)
    ignore_players.append(lineups[0].name.sample(1).iloc[0])
    print(ignore_players)

deque(['Derrick Henry'], maxlen=2)


<ipython-input-194-23f7b9af2c8e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lineup["lineup_id"] = uuid4().hex


deque(['Derrick Henry', 'Diontae Johnson'], maxlen=2)
deque(['Diontae Johnson', 'Diontae Johnson'], maxlen=2)
deque(['Diontae Johnson', 'Daniel Jones'], maxlen=2)
deque(['Daniel Jones', 'James Washington'], maxlen=2)
deque(['James Washington', 'Evan Engram'], maxlen=2)
deque(['Evan Engram', 'Derrick Henry'], maxlen=2)
deque(['Derrick Henry', 'JuJu Smith-Schuster'], maxlen=2)
deque(['JuJu Smith-Schuster', 'Diontae Johnson'], maxlen=2)
deque(['Diontae Johnson', 'Evan Engram'], maxlen=2)


In [195]:
lineups = pd.concat(lineups, ignore_index=True)

In [196]:
lineups.to_csv("../data/draftkings/lineups/20200914_56k.csv", index=False)

In [191]:
lineups = pd.read_csv("../data/draftkings/lineups/20200914_55k.csv")
with pd.option_context("display.max_rows", None):
    display(lineups[["lineup_id", "name", "position", "team_name"]])

,lineup_id,name,position,team_name
0,a887c30e488245b698c7251ad00bd8b4,Eric Ebron,TE,Pittsburgh Steelers
1,a887c30e488245b698c7251ad00bd8b4,Melvin Gordon,RB,Denver Broncos
2,a887c30e488245b698c7251ad00bd8b4,Derrick Henry,RB,Tennessee Titans
3,a887c30e488245b698c7251ad00bd8b4,JuJu Smith-Schuster,WR,Pittsburgh Steelers
4,a887c30e488245b698c7251ad00bd8b4,James Washington,WR,NaN
5,a887c30e488245b698c7251ad00bd8b4,Saquon Barkley,RB,New York Giants
6,a887c30e488245b698c7251ad00bd8b4,Diontae Johnson,WR,Pittsburgh Steelers
7,a887c30e488245b698c7251ad00bd8b4,Daniel Jones,QB,New York Giants
8,9f4ec314ece149a7a9d12549e15d795c,Melvin Gordon,RB,Denver Broncos
9,9f4ec314ece149a7a9d12549e15d795c,Derrick Henry,RB,Tennessee Titans


In [197]:
lineups = pd.read_csv("../data/draftkings/lineups/20200914_56k.csv")
with pd.option_context("display.max_rows", None):
    display(lineups[["lineup_id", "name", "position", "team_name"]])

,lineup_id,name,position,team_name
0,742eb7ce42454d1db3f2e19354dd91ab,Melvin Gordon,RB,Denver Broncos
1,742eb7ce42454d1db3f2e19354dd91ab,Derrick Henry,RB,Tennessee Titans
2,742eb7ce42454d1db3f2e19354dd91ab,JuJu Smith-Schuster,WR,Pittsburgh Steelers
3,742eb7ce42454d1db3f2e19354dd91ab,Evan Engram,TE,New York Giants
4,742eb7ce42454d1db3f2e19354dd91ab,James Washington,WR,NaN
5,742eb7ce42454d1db3f2e19354dd91ab,Saquon Barkley,RB,New York Giants
6,742eb7ce42454d1db3f2e19354dd91ab,Diontae Johnson,WR,Pittsburgh Steelers
7,742eb7ce42454d1db3f2e19354dd91ab,Daniel Jones,QB,New York Giants
8,d4093dde87ff498695a2c271e5d1cf4f,Melvin Gordon,RB,Denver Broncos
9,d4093dde87ff498695a2c271e5d1cf4f,JuJu Smith-Schuster,WR,Pittsburgh Steelers


In [109]:
lineups = pd.read_csv("../data/draftkings/lineups/20200913_57k.csv")
lineups[["lineup_id", "name", "position"]]

,lineup_id,name,position
0,426e90aaf4a84e478dafec2fce447b85,DeAndre Hopkins,WR
1,426e90aaf4a84e478dafec2fce447b85,Keenan Allen,WR
2,426e90aaf4a84e478dafec2fce447b85,Todd Gurley,RB
3,426e90aaf4a84e478dafec2fce447b85,Chris Conley,WR
4,426e90aaf4a84e478dafec2fce447b85,Carson Wentz,QB
5,426e90aaf4a84e478dafec2fce447b85,Kenyan Drake,RB
6,426e90aaf4a84e478dafec2fce447b85,Christian McCaffrey,RB
7,426e90aaf4a84e478dafec2fce447b85,Mike Gesicki,TE
8,374cc647162244abb997307ffe7ea052,Julio Jones,WR
9,374cc647162244abb997307ffe7ea052,Keenan Allen,WR
